<a href="https://colab.research.google.com/github/unt-iialab/Legal-KG/blob/master/Legal-IE/Extraction/entity_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install lexnlp

     |████████████████████████████████| 9.7MB 2.9MB/s 
     |████████████████████████████████| 10.3MB 41.8MB/s 
     |████████████████████████████████| 614kB 44.1MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
     |████████████████████████████████| 358kB 55.2MB/s 
     |████████████████████████████████| 102kB 11.5MB/s 
     |████████████████████████████████| 50.0MB 76kB/s 
     |████████████████████████████████| 235kB 42.6MB/s 
     |████████████████████████████████| 8.9MB 52.2MB/s 
     |████████████████████████████████| 6.7MB 39.5MB/s 
     |████████████████████████████████| 61kB 7.6MB/s 
     |████████████████████████████████| 1.5MB 49.8MB/s 
     |████████████████████████████████| 22.6MB 76.1MB/s 
     |████████████████████████████████| 133kB 44.3MB/s 
  Created wheel for regex: filename=regex-2017.9.23-cp36-cp36m-linux_x86_64.whl size=538792 sha256=4c7af9df89f24a151de7275192909e28ae40b0a41156ef1b18fca55c0f884397
  Stored in directory: /root/.cache/pip/wheels/57/19/41/

In [0]:
import lexnlp.nlp.en.segments.sentences
import nltk
import os

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
import re

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
# Split the documents into sentences

path = '/content/data'
for doc in os.listdir(path):
  print(doc)
  doc_path = os.path.join(path,doc)
  with open(doc_path,'r') as file:
   content = file.read()
   #print(content)
   sent = lexnlp.nlp.en.segments.sentences.get_sentence_list(content)
   #print(sent)
   with open('/content/process_data/'+doc,'w') as outfile:
     for listitem in sent:
       outfile.write('%s\n'%listitem)

01-05-1  Adams v Tanner.txt
01-05-2  Ex parte GWW.txt
01-05-4  Jackson v Millspaugh.txt
01-05-3  Graham v King.txt
01-05-5  Jacoby v Funkhouser.txt


In [0]:
# remove some noise by defining a set of patterns

path = '/content/process_data'
for doc in os.listdir(path):
  print(doc)
  doc_path = os.path.join(path,doc)
  with open(doc_path,'r') as f:
    lines = f.read().split('\n')
    for i,line in enumerate(lines):
      #print(i,line)
      if line.startswith('v.'):
        lines[i-1] = lines[i-1] +lines[i]
        lines.pop(i)
        for j, l in enumerate(lines):
          #print(j,l)
          if l.endswith('v.'):
            lines[j] = lines[j] + lines[j+1]
            lines.pop(j+1)
          else:
            pass

      elif line.endswith('v.'):
        lines[i] = lines[i] +lines[i+1]
        lines.pop(i+1)

      if line[:3].isdigit():
        lines[i-1] = lines[i-1] + lines[i]
        #print(line)
        lines.pop(i)

        for j,l in enumerate(lines):
          if l[:3].isdigit():
            #print(l)
            lines[j-1] = lines[j-1] + lines[j]
            lines.pop(j)
          else:
            pass
    
      if line.endswith(':'):
        #print(line)
        lines[i] = lines[i] + lines[i+1]
        lines.pop(i+1)

  with open('/content/final_clean/'+doc,'w') as outf:
      for items in lines:
        outf.write('%s\n'%items)

01-05-1  Adams v Tanner.txt
01-05-2  Ex parte GWW.txt
01-05-4  Jackson v Millspaugh.txt
01-05-3  Graham v King.txt
01-05-5  Jacoby v Funkhouser.txt


In [0]:
# entity extraction
import pandas
import lexnlp.extract.en.amounts
import lexnlp.extract.en.acts
import lexnlp.extract.en.citations
import lexnlp.extract.en.courts
import lexnlp.extract.en.dict_entities
import lexnlp.extract.en.entities.nltk_re
import lexnlp.extract.en.conditions
import lexnlp.extract.en.copyright
import lexnlp.extract.en.cusip
import lexnlp.extract.en.dates
import lexnlp.extract.en.definitions
import lexnlp.extract.en.distances
import lexnlp.extract.en.durations
import lexnlp.extract.en.geoentities
import lexnlp.extract.en.money
import lexnlp.extract.en.percents
import lexnlp.extract.en.pii
import lexnlp.extract.en.ratios
import lexnlp.extract.en.regulations
import lexnlp.extract.en.trademarks
import lexnlp.extract.en.urls
import lexnlp.extract.en.entities.nltk_maxent
import lexnlp.extract.en.entities.stanford_ner
import lexnlp.extract.en.addresses.addresses


court_df = pandas.read_csv("https://raw.githubusercontent.com/LexPredict/lexpredict-legal-dictionary/1.0.5/en/legal/us_courts.csv")
court_config_data = []
for _, row in court_df.iterrows():
    c = lexnlp.extract.en.dict_entities.entity_config(row["Court ID"], row["Court Name"], 0, row["Alias"].split(";") if not pandas.isnull(row["Alias"]) else [])
    court_config_data.append(c)

path = '/content/final_clean'
for doc in os.listdir(path):
  print(doc)
  doc_path = os.path.join(path,doc)
  with open(doc_path,'r') as f:
    lines = f.read().split('\n')
    for line in lines:
      # extract amount by using get_amount
      amounts = list(lexnlp.extract.en.amounts.get_amounts(line))
      # if len(amounts)>0:
      #   print (amounts)

      # extract acts by using get_act_list()
      acts = lexnlp.extract.en.acts.get_act_list(line)
      # if len(acts)>0:
      #   print(acts)

      # extract citations by using get_citations()
      citations = list(lexnlp.extract.en.citations.get_citations(line))
      if len(citations)>0:
        print(citations)

      # extract courts by using get_courts()
      for entity, alias in lexnlp.extract.en.courts.get_courts(line, court_config_data):
        print("entity=", entity)
        print("alias=", alias)
      
      # extract companies by using nltk_re.get_entities.nltk_re.get_companies()
      companies = list(lexnlp.extract.en.entities.nltk_re.get_entities.nltk_re.get_companies(line))
      # companies = list(lexnlp.extract.en.entities.nltk_re.get_entities.nltk_re(text))

      # extract conditions by using get_conditions()
      conditions = list(lexnlp.extract.en.conditions.get_conditions(line))

      # extract constraints by using get_constraints()
      constraints = list(lexnlp.extract.en.constraints.get_constraints(line))

      # extract copyright by using get_copyright()
      copyrights = list(lexnlp.extract.en.copyright.get_copyright(line))

      # extract cusip by using get_cusip()
      cusips = lexnlp.extract.en.cusip.get_cusip(line)

      # extract dates by using get_dates()
      dates = list(lexnlp.extract.en.dates.get_dates(line))

      # extract definitions by using get_definitions()
      definitions = list(lexnlp.extract.en.definitions.get_definitions(line))

      # extract distances by using get_distances()
      distances = list(lexnlp.extract.en.distances.get_distances(line))

      # extract durations by using get_durations()
      durations = list(lexnlp.extract.en.durations.get_durations(line))

      # extract geoentities by using get_geoentities()
      geoentities = lexnlp.extract.en.geoentities.get_geoentities(line)


      # extract money by using get_money()
      moneys = list(lexnlp.extract.en.money.get_money(line))

      # extract percents by using .get_percents()
      percents = list(lexnlp.extract.en.percents.get_percents(line))

      # extract all the piis by using get_pii()
      piis = list(lexnlp.extract.en.pii.get_pii(line))

      # extract all the ratios by using get_ratios()
      ratios = list(lexnlp.extract.en.ratios.get_ratios(line))

      # extract all the regulations by using get_regulations() 
      regulations = list(lexnlp.extract.en.regulations.get_regulations(line))

      # extract all the trademarks by using get_trademarks()
      trademarks = list(lexnlp.extract.en.trademarks.get_trademarks(line))

      # extract all the urls by using get_urls()
      urls = list(lexnlp.extract.en.urls.get_urls(line))

      # extract all the persons by using entities.nltk_maxent.get_persons()
      persons = list(lexnlp.extract.en.entities.nltk_maxent.get_persons(line))
      # persons = list(lexnlp.extract.en.entities.stanford_ner.get_persons(line))

      # extract all the geopolitical by using entities.nltk_maxent.get_geopolitical()
      geopolitical = list(lexnlp.extract.en.entities.nltk_maxent.get_geopolitical(line))

      # extract noun_phrases
      noun_phrases = list(lexnlp.extract.en.entities.nltk_maxent.get_noun_phrases(line))

      # extract organizations
      organizations = list(lexnlp.extract.en.entities.stanford_ner.get_organizations(line))

      # extract locations
      locations = list(lexnlp.extract.en.entities.stanford_ner.get_locations(line))

      # extract address 
      addresses = list(lexnlp.extract.en.addresses.addresses.get_addresses(line))




01-05-1  Adams v Tanner.txt
entity= (1, 'United States Supreme Court', 0, [('United States Supreme Court', None, False, None, ' united states supreme court '), ('Supreme Court', None, False, None, ' supreme court '), (' SCOTUS', None, False, None, ' scotus ')])
alias= ('Supreme Court', None, False, None, ' supreme court ')
[(1843, 'WL', 'West Law Citation', 284, None, None, None)]
[(8, 'Cal.', 'California Reports', 145, '147', None, None)]
01-05-2  Ex parte GWW.txt
entity= (1, 'United States Supreme Court', 0, [('United States Supreme Court', None, False, None, ' united states supreme court '), ('Supreme Court', None, False, None, ' supreme court '), (' SCOTUS', None, False, None, ' scotus ')])
alias= ('Supreme Court', None, False, None, ' supreme court ')
[(2014, 'WL', 'West Law Citation', 11352677, None, None, None)]
entity= (1, 'United States Supreme Court', 0, [('United States Supreme Court', None, False, None, ' united states supreme court '), ('Supreme Court', None, False, None, 